# Chapter 12: Distributing TensorFlow Across Devices and Servers

This notebook is my solution to exercise 10 of chapter 12. It contains three distributed models. Each model requires you restart the kernel and run the code in the **Installation** section.

## Exercise 10

Train a DNN using between-graph replication and data parallelism with asynchronous updates, timing how long it takes to reach a satisfying performance. Next, try again using synchronous updates. Do synchronous updates produce a better model? Does it train faster? Split the DNN vertically and place each vertical slice on a different device, and train the model again. Is training any faster? Is performance any different?

## Solution

### Installation

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!pip3 install --upgrade tensorflow-gpu

### Asynchronous Updates

Since I am not able to run TensorFlow from multiple clients in Colab, I am going to be using in-graph replication instead for asynchronous updates.

In [0]:
# Downloading MNIST dataset.

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Defining the cluster spec for the parallel model.

n_dnns = 3

cluster_spec = tf.train.ClusterSpec({
    'ps': ['127.0.0.1:1000'],
    'worker': ['127.0.0.1:100{}'.format(i + 1) for i in range(n_dnns)]
})

In [0]:
# Abstracting the operations with the individual workers which train their own
# copy of the DNN into a class. This model uses the hyperparameters that led
# to the best performance on the validation set in exercise 8.

class DNNTask:
  def __init__(self, X, y, task, parameters, activation=tf.nn.elu, n_inputs=(28 ** 2),
               n_outputs=10, learning_rate=0.01, momentum=0.95, n_epochs=100,
               batch_size=50):
    self.task = task
    self.parameters = parameters
    self.batch_size = batch_size
    self.n_batches = len(X_train) // batch_size
    self.n_epochs = n_epochs

    self._gpu_name = '/job:worker/task:{}/gpu:0'.format(task)
    self._cpu_name = '/job:worker/task:{}/cpu:0'.format(task)
    self._X = X
    self._y = y

    with tf.device(self._gpu_name):
      with tf.variable_scope('worker{}'.format(task)):
        self._hidden_layers = []
        self._grad_vars = []
        for i, params in enumerate(parameters[:-1]):
          W, b = params
          self._grad_vars += [W, b]
          self._hidden_layers.append(
              activation(
                  tf.matmul(
                      (self._X if i == 0 else self._hidden_layers[-1]), W) + b))
        W, b = parameters[-1]
        self._logits = tf.matmul(self._hidden_layers[-1], W) + b
        self._xentropy = \
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self._y,
                                                           logits=self._logits)
        self._loss = tf.reduce_mean(self._xentropy)
        self._optimizer = \
            tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=momentum)
        self._training_op = self._optimizer.minimize(self._loss,
                                                     var_list=parameters)

    with tf.device(self._cpu_name):
      with tf.variable_scope('worker{}'.format(task)):
        self._init = tf.global_variables_initializer()
        self._correct = tf.nn.in_top_k(self._logits, self._y, 1)
        self._accuracy = tf.reduce_mean(tf.cast(self._correct, tf.float32))
        self._shuffle_queue = \
            tf.RandomShuffleQueue(capacity=len(X_train), min_after_dequeue=0,
                                  dtypes=[tf.float32, tf.int32],
                                  shapes=[(n_inputs), ()], name='input_queue',
                                  shared_name='input_queue')
        self._enqueue_op = self._shuffle_queue.enqueue_many([self._X, self._y],
                                                            name='enqueue')
        self._dequeue_op = self._shuffle_queue.dequeue_up_to(batch_size,
                                                             name='dequeue')

  def train_model(self, sess):
    with sess.as_default():
      sess.run(self._init)

      for epoch in range(self.n_epochs):
        sess.run(self._enqueue_op, feed_dict={self._X: X_train,
                                               self._y: y_train})
        for _ in range(self.n_batches):
          X_batch, y_batch = sess.run(self._dequeue_op)
          sess.run(self._training_op, feed_dict={self._X: X_batch,
                                                 self._y: y_batch})

In [0]:
# Defining the graph for the model.

n_inputs = 28 ** 2
n_hidden_layers = 5
n_neurons = 160
stddev = 2.0 / np.sqrt(n_inputs + n_neurons)
n_outputs = 10

tf.reset_default_graph()
ensemble = []

with tf.device('/job:ps/task:0/cpu:0'):
  with tf.variable_scope('ps0'):
    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
    y = tf.placeholder(tf.int32, shape=(None), name='y')

    parameters = []
    for i in range(n_hidden_layers):
      W = tf.Variable(
          tf.truncated_normal(
              (n_inputs if i == 0 else n_neurons, n_neurons),
              mean=0.0, stddev=stddev))
      b = tf.Variable(tf.zeros([n_neurons]))
      parameters.append((W, b))
    W = tf.Variable(
        tf.truncated_normal((n_neurons, n_outputs),
            mean=0.0, stddev=stddev))
    b = tf.Variable(tf.zeros([n_outputs]))
    parameters.append((W, b))

    hidden_layers = []
    for i, params in enumerate(parameters[:-1]):
      W, b = params
      hidden_layers.append(
          tf.nn.elu(tf.matmul((X if i == 0 else hidden_layers[-1]), W) + b))
    W, b = parameters[-1]
    logits = tf.matmul(hidden_layers[-1], W) + b
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()

for task in range(n_dnns):
  ensemble.append(DNNTask(X, y, task, parameters))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Starting the servers.

ps = tf.train.Server(cluster_spec, job_name='ps', task_index=0)
workers = []

for task in range(n_dnns):
  workers.append(
      tf.train.Server(cluster_spec, job_name='worker', task_index=task))

In [0]:
# Defining the Clock class for timing training.

import time

class Clock:
  def __init__(self):
    self.start_time = None
  def start(self):
    self.start_time = time.time()
    return self
  def stop(self):
    dt = time.time() - self.start_time
    self.start_time = None
    h, m, s = int(dt // 3600), int(dt % 3600) // 60, dt % 60
    return '{}h {}m {:.3f}s'.format(h, m, s)

In [0]:
# Running the training algorithm

from threading import Thread

clock = Clock().start()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(ps.target, config=config) as sess:
  sess.run(init)
  threads = []
  for task in range(n_dnns):
    thread = Thread(target=lambda s: ensemble[task].train_model(s),
                    args=(sess,))
    thread.start()
    threads.append(thread)
  for thread in threads:
    thread.join()
  acc_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
print('Time taken to train model:', clock.stop())
print('Test set accuracy', acc_val)

Time taken to train model: 0h 43m 50.663s
Test set accuracy 0.9832


The model performed about as well as a single neural network with those parameters, but training took significantly longer.

### Synchronous Updates

TODO using TensorFlow's experimental `Strategry` API after going through their guide.